In [1]:
import cv2
import numpy as np
from easydict import EasyDict

root = "dataset\\kitti\\"
raw = root + "object\\training\\"
img2 = raw + "image_2\\"
img3 = raw + "image_3\\"
lidar = raw + "velodyne\\"
calib = raw + "calib\\"
label = raw + "label_2\\"

sample_id = 6
image_name = f"{sample_id:06d}.png"
lidar_name = f"{sample_id:06d}.bin"
calib_name = f"{sample_id:06d}.txt"
label_name = f"{sample_id:06d}.txt"

boundary = EasyDict({
    "minX": 0,
    "maxX": 50,
    "minY": -25,
    "maxY": 25,
    "minZ": -2.73,
    "maxZ": 1.27
})
BEV_WIDTH = 608   # across y axis -25m ~ 25m
BEV_HEIGHT = 608  # across x axis 0m ~ 50m
DISCRETIZATION_X = (boundary.maxX - boundary.minX) / BEV_HEIGHT
DISCRETIZATION_Y = (boundary.maxY - boundary.minY) / BEV_HEIGHT

In [18]:
# Image
cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)
cim3 = cv2.imread(img3+image_name, cv2.IMREAD_COLOR)
# print(type(cim2))
print(cim2.shape)

# axis 0: append below
# asix 1: append next
cv2.imwrite(f"{sample_id:06d}_camera.png", cim2) # np.concatenate((cim2, cim3), axis=0))
cv2.imshow(image_name, np.concatenate((cim2, cim3), axis=0))
cv2.waitKey(0)
cv2.destroyAllWindows()

(374, 1238, 3)


In [14]:
def rgb(lidar, lidar_name):
    # lidar
    lidar_data = np.fromfile(lidar+lidar_name, dtype=np.float32).reshape(-1, 4)

    # remove boundary
    # x, y, z = 0, 1, 2
    mask = np.where(
        (lidar_data[:, 0] >= boundary.minX) &
        (lidar_data[:, 0] <= boundary.maxX) &

        (lidar_data[:, 1] >= boundary.minY) &
        (lidar_data[:, 1] <= boundary.maxY) &

        (lidar_data[:, 2] >= boundary.minZ) &
        (lidar_data[:, 2] <= boundary.maxZ)
    )
    lidar_data = lidar_data[mask]
    lidar_data[:, 2] -= boundary.minZ

    # make BEV feature
    height = BEV_HEIGHT + 1  # NOTE: why?
    width = BEV_WIDTH + 1  # ?

    # discretize
    bev = np.copy(lidar_data)
    bev[:, 0] = np.int_(np.floor(bev[:, 0] / DISCRETIZATION_X))
    bev[:, 1] = np.int_(np.floor(bev[:, 1] / DISCRETIZATION_Y) + width / 2)

    # x, y, -z로 정렬
    indices = np.lexsort((-bev[:, 2], bev[:, 1], bev[:, 0]))
    bev = bev[indices]

    height_map = np.zeros((height, width))
    intensity_map = np.zeros((height, width))
    density_map = np.zeros((height, width))

    _, indices, counts = np.unique(
        bev[:, 0:2], axis=0, return_index=True, return_counts=True)
    bev_top = bev[indices]

    # height
    max_height = float(np.abs(boundary.maxZ - boundary.minZ))
    height_map[np.int_(bev_top[:, 0]), np.int_(
        bev_top[:, 1])] = bev_top[:, 2] / max_height

    # intensity
    intensity_map[np.int_(bev_top[:, 0]), np.int_(bev_top[:, 1])] = bev_top[:, 3]

    # density
    # count수를 log_64 스케일로 변환
    norm_counts = np.minimum(1.0, np.log(counts + 1) / np.log(64))
    density_map[np.int_(bev_top[:, 0]), np.int_(bev_top[:, 1])] = norm_counts

    # height - 1, width - 1 = BEV_HEIGHT, BEV_WIDTH
    rgb_map = np.zeros((3, height - 1, width - 1))
    rgb_map[2, :, :] = density_map[:BEV_HEIGHT, :BEV_WIDTH]     # r_map
    rgb_map[1, :, :] = height_map[:BEV_HEIGHT, :BEV_WIDTH]      # g_map
    rgb_map[0, :, :] = intensity_map[:BEV_HEIGHT, :BEV_WIDTH]   # b_map

    # test = x[0].detach().to('cpu').numpy()
    rgb_map_transpose = np.transpose(rgb_map, (1, 2, 0))
    # rgb_map_transpose = rgb_map_transpose / rgb_map_transpose.max() * 255
    # rgb_map_transpose = rgb_map_transpose.astype(np.uint8)
    # cv2.imwrite(f"{raw}\\rgb_2\\{sample_id:06d}.png", rgb_map_transpose*255)
    # cv2.imwrite(f"{sample_id:06d}_lidar.png", rgb_map_transpose*255)
    cv2.imshow(lidar_name, rgb_map_transpose)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return rgb_map_transpose.copy()

binimg = rgb(lidar, lidar_name)
print(binimg.shape, binimg.dtype)
# for sample_id in range(0, 7480+1):
#     lidar_name = f"{sample_id:06d}.bin"
#     rgb(lidar, lidar_name)

(608, 608, 3) float64


In [13]:
# IPM
import cv2
import math

# read cam2
cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)

DEG2RAD = 0.01745329252
CAMERA_POS_Y = 0   # d (cm)          # 173
CAMERA_POS_X = 0  # l (cm)          # -6
CAMERA_POS_Z = 173 # 60 # h (cm)    # 168
FOV_H = 45.0      # (degree)
FOV_V = 55.0      # (degree)

def build_ipm_table(
    srcw: int, srch: int, dstw: int, dsth: int,
    vptx: int, vpty: int):
    maptable = np.zeros((dstw*dsth,), dtype=int)
    
    alpha_h = 0.5 * FOV_H * DEG2RAD
    alpha_v = 0.5 * FOV_V * DEG2RAD
    gamma = -(vptx - (srcw >> 1)) * alpha_h / (srcw >> 1) # camera pan angle
    theta = -(vpty - (srch >> 1)) * alpha_v / (srch >> 1) # camera tilt angle

    front_map_start_position = dsth >> 1
    front_map_end_position = front_map_start_position + dsth
    side_map_mid_position = dstw >> 1

    # scale to get better mapped image
    front_map_scale_factor = 4
    side_map_scale_factor  = 2

    for y in range(dstw):
        for x in range(front_map_start_position, front_map_end_position):
            idx = y * dsth + (x - front_map_start_position)
            deltax = front_map_scale_factor * (front_map_end_position - x - CAMERA_POS_X)
            deltay = side_map_scale_factor * (y - side_map_mid_position - CAMERA_POS_Y)

            if deltay == 0:
                maptable[idx] = maptable[idx - dsth]
            else:
                atan_deltay_deltax = math.atan(deltay / deltax)
                u = (int)(math.atan(CAMERA_POS_Z * math.sin(atan_deltay_deltax / deltay) - (theta - alpha_v)) / (2 * alpha_v / srch))
                v = (int)((atan_deltay_deltax - (gamma - alpha_h)) / (2 * alpha_h / srcw))
                if 0 <= u < srch and 0 <= v < srcw:
                    maptable[idx] = srcw * u + v
                else:
                    maptable[idx] = -1
    return maptable

def inverse_perspective_mapping(
    dstw: int, dsth: int, src, maptable: list):
    dst = np.zeros((dsth*dstw, 3), dtype = "uint8")
    src = src.reshape((-1, 3))
    # dst image (1cm/pixel)
    idx = 0
    for j in range(dsth):
        for i in range(dstw):
            if maptable[idx] != -1:
                dst[i * dsth + j] = src[maptable[idx]]
            else:
                dst[i * dsth + j] = [0, 0, 0]
            idx+=1
    return dst

SRC_RESIZED_WIDTH   = cim2.shape[1]
SRC_RESIZED_HEIGHT  = cim2.shape[0]
DST_REMAPPED_WIDTH  = 608
DST_REMAPPED_HEIGHT = 608

# init vanishing point at center of image
vanishing_point_x = SRC_RESIZED_WIDTH  >> 1
vanishing_point_y = SRC_RESIZED_HEIGHT >> 1

# build inverse perspective mapping table first
ipm_table = build_ipm_table(SRC_RESIZED_WIDTH, SRC_RESIZED_HEIGHT, 
                           DST_REMAPPED_WIDTH, DST_REMAPPED_HEIGHT, 
                           vanishing_point_x, vanishing_point_y)

# cv2.resize(img_bev, (configs.img_size, configs.img_size))
cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)
# cim2_res = cv2.resize(cim2, (SRC_RESIZED_WIDTH, SRC_RESIZED_HEIGHT))
# cim2_gray = cv2.cvtColor(cim2_res, CV_BGR2GRAY)

img_mapped = inverse_perspective_mapping(DST_REMAPPED_WIDTH, DST_REMAPPED_HEIGHT, cim2, ipm_table)
img_mapped = img_mapped.reshape((DST_REMAPPED_HEIGHT, DST_REMAPPED_WIDTH, 3))
img_mapped = cv2.rotate(img_mapped, cv2.ROTATE_180)

# line(imresize, Point(vanishing_point_x + 10, vanishing_point_y), Point(vanishing_point_x - 10, vanishing_point_y), Scalar(0, 0, 255));
# line(imresize, Point(vanishing_point_x, vanishing_point_y + 10), Point(vanishing_point_x, vanishing_point_y - 10), Scalar(0, 0, 255));
# imshow("resize", imresize);
cv2.imshow("remap", img_mapped); cv2.waitKey(0); cv2.destroyAllWindows()
cv2.imwrite(f'{sample_id:06d}_ipm.png', img_mapped)
# binimg = img_mapped # Overlap

In [16]:
# binimg
import cv2
import yaml
from src.data_process.kitti_data_utils import Object3d


# read label data
with open(label+label_name, 'r') as lfile:
    lines = [line.rstrip() for line in lfile]
    objects = [Object3d(line) for line in lines]
# read calib data
with open(calib+calib_name, 'r') as cfile:
    calib_dict = yaml.load(cfile, Loader=yaml.FullLoader)
calib_dict = {key: [float(v) for v in value.split(' ')]
              for key, value in calib_dict.items()}

bbox_selected = []
for obj in objects:
    if obj.cls_id == -1:
        continue
    bbox = []
    bbox.append(obj.cls_id)
    bbox.extend([obj.t[0], obj.t[1], obj.t[2], obj.h, obj.w, obj.l, obj.ry])
    bbox_selected.append(bbox)
labels = np.array(bbox_selected, dtype=np.float32)

# cam to lidar_box
ret = []
for box in labels[:, 1:]:
    x, y, z, h, w, l, ry = box

    # camera to lidar
    p = np.array([x, y, z, 1])
    R0_i = np.eye(4)
    R0_i[:3, :3] = np.array(
        list(map(float, calib_dict['R0_rect']))).reshape(3, 3)
    p = np.matmul(np.linalg.inv(R0_i), p)
    tr = np.eye(4)
    tr[:3, :4] = np.array(
        list(map(float, calib_dict['Tr_velo_to_cam']))).reshape(3, 4)
    inv_Tr = np.zeros_like(tr)  # 3x4
    inv_Tr[0:3, 0:3] = np.transpose(tr[0:3, 0:3])
    inv_Tr[0:3, 3] = np.dot(-np.transpose(tr[0:3, 0:3]), tr[0:3, 3])
    p = np.matmul(inv_Tr, p)
    p = p[0:3]
    x, y, z = tuple(p)

    h, w, l, rz = h, w, l, (-ry - np.pi / 2)
    # rz = angle_in_limit(rz)
    ret.append([x, y, z, h, w, l, rz])
labels[:, 1:] = np.array(ret).reshape(-1, 7)

minX, maxX = 0, 50
minY, maxY = -25, 25
minZ, maxZ = -2.73, 1.27
colors = [[0, 255, 255], [0, 0, 255], [255, 0, 0]]

targets = []
for i in range(labels.shape[0]):
    cl, x, y, z, h, w, l, yaw = labels[i]
    # ped and cyc labels are very small, so lets add some factor to height/width
    # l = l + 0.3
    # w = w + 0.3
    yaw = np.pi * 2 - yaw
    if (minX < x < maxX) and (minY < y < maxY):
        # we should put this in [0,1], so divide max_size  80 m
        y1 = (y - minY) / (maxY - minY)
        # we should put this in [0,1], so divide max_size  40 m
        x1 = (x - minX) / (maxX - minX)
        w1 = w / (maxY - minY)
        l1 = l / (maxX - minX)
        # target.append([cl, y1, x1, w1, l1, math.sin(float(yaw)), math.cos(float(yaw))])
        targets.append([cl, y1, x1, w1, l1, yaw])
targets = np.array(targets, dtype=np.float32)


def get_corners(x, y, w, l, yaw):
    bev_corners = np.zeros((4, 2), dtype=np.float32)
    cos_yaw = np.cos(yaw)
    sin_yaw = np.sin(yaw)
    # front left
    bev_corners[0, 0] = x - w / 2 * cos_yaw - l / 2 * sin_yaw
    bev_corners[0, 1] = y - w / 2 * sin_yaw + l / 2 * cos_yaw

    # rear left
    bev_corners[1, 0] = x - w / 2 * cos_yaw + l / 2 * sin_yaw
    bev_corners[1, 1] = y - w / 2 * sin_yaw - l / 2 * cos_yaw

    # rear right
    bev_corners[2, 0] = x + w / 2 * cos_yaw + l / 2 * sin_yaw
    bev_corners[2, 1] = y + w / 2 * sin_yaw - l / 2 * cos_yaw

    # front right
    bev_corners[3, 0] = x + w / 2 * cos_yaw - l / 2 * sin_yaw
    bev_corners[3, 1] = y + w / 2 * sin_yaw + l / 2 * cos_yaw

    return bev_corners


# binimg = np.zeros((608, 608, 3)) # delete if you need overlap
targets[:, 1:5] *= 608
print(f"{targets.shape=}")
# target[:, 5]
for cls_id, x, y, w, l, yaw in targets:
    color = colors[int(cls_id)]
    bev_corners = get_corners(x, y, w, l, yaw)
    corners_int = bev_corners.reshape(-1, 1, 2).astype(int)
    cv2.fillPoly(binimg, [corners_int], color)
# cv2.imwrite(f'{sample_id:06d}_binimg.png', binimg)
cv2.imshow('binimg', binimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

targets.shape=(4, 6)


In [27]:
# # lidar to image
# import matplotlib.pyplot as plt
# with open(calib+calib_name, 'r') as cfile:
#     calib_data = cfile.read()
# calib_data = calib_data.strip()
# calib_data = calib_data.split('\n')
# # print(calib_data)

# calib_dict = {}
# for cd in calib_data:
#     sp = cd.split(":")
#     calib_dict[sp[0]] = sp[1].strip().split(" ")
# # calib_dict

# # read image
# # cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)
# # cim3 = cv2.imread(img3+image_name, cv2.IMREAD_COLOR)

# # read lidar
# lidar_data = np.fromfile(lidar+lidar_name, dtype=np.float32).reshape(-1, 4)
# # TODO find boundary
# # mask = np.where(
# #     (lidar_data[:,0] >= boundary.minX) &
# #     (lidar_data[:,0] <= boundary.maxX) &

# #     (lidar_data[:,1] >= boundary.minY) &
# #     (lidar_data[:,1] <= boundary.maxY) &

# #     (lidar_data[:,2] >= boundary.minZ) &
# #     (lidar_data[:,2] <= boundary.maxZ)
# # )
# # lidar_data = lidar_data[mask]
# # xyz = lidar_data[:,:3]

# # mapping to image
# p2 = np.array(list(map(float, calib_dict['P2']))).reshape(3, 4)
# # print(f"{p2=}, {p2.shape}")
# p3 = np.array(list(map(float, calib_dict['P3']))).reshape(3, 4)

# r0 = np.eye(4)
# r0[:3, :3] = np.array(list(map(float, calib_dict['R0_rect']))).reshape(3, 3)
# # print(f"{r0=}, {r0.shape}")

# tr = np.eye(4)
# tr[:3, :4] = np.array(
#     list(map(float, calib_dict['Tr_velo_to_cam']))).reshape(3, 4)
# # print(f"{tr=}, {tr.shape}")

# np.set_printoptions(formatter={'float_kind': "{:.4f}".format})
# print(f"{r0@tr=}")

# # get calib
# xy2 = ((p2 @ r0) @ tr) @ lidar_data.transpose()
# print((p2@r0).shape)
# xy2 = xy2.transpose()
# print(f"{xy2.shape=}")

# # b_ones = np.ones((1,3), int)

# s2 = xy2[:, 2]  # z
# x2 = xy2[:, 0] / s2
# y2 = xy2[:, 1] / s2

# plt.figure(figsize=(12, 4), dpi=100)
# plt.plot(x2, y2, 'r.', markersize=0.5)
# # plt.gca().invert_yaxis()
# # (375, 1242, 3)
# plt.xlim(0, 1242)
# plt.ylim(375, 0)
# plt.show()

# # cam_map = np.zeros((1, BEV_HEIGHT, BEV_WIDTH))
# # cam_map[:, 0, :] = x2
# # cam_map[:, :, 0] = y2
# # cam_map = np.transpose(cam_map, (1,2,0))
# # cv2.imshow("cam map", cam_map); cv2.waitKey(0); cv2.destroyAllWindows();

In [77]:
import cv2
import yaml

# read calib data
with open(calib+calib_name, 'r') as cfile:
    calib_dict = yaml.load(cfile, Loader=yaml.FullLoader)
calib_dict = {key: [float(v) for v in value.split(' ')]
              for key, value in calib_dict.items()}
# read lidar
lidar_data = np.fromfile(lidar+lidar_name, dtype=np.float32).reshape(-1, 4)
lidar_order= np.arange(lidar_data.shape[0])
# read cam2
cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)

# mapping to image
p2 = np.array(list(map(float, calib_dict['P2']))).reshape(3, 4)
# print(f"{p2=}, {p2.shape}")
r0 = np.eye(4)
r0[:3,:3] = np.array(list(map(float, calib_dict['R0_rect']))).reshape(3, 3)
# print(f"{r0=}, {r0.shape}")
tr = np.eye(4)
tr[:3,:4] = np.array(list(map(float, calib_dict['Tr_velo_to_cam']))).reshape(3, 4)
# print(f"{tr=}, {tr.shape}")

## get calib
xy2 = ((p2 @ r0) @ tr) @ lidar_data.transpose()
xy2 = xy2.transpose()
print(f"{xy2.shape=}")

# b_ones = np.ones((1,3), int)
H, W = cim2.shape[0], cim2.shape[1]
# projected = np.ones((H, W, 3), dtype=np.uint8) * 255 # white
projected = cim2 # image

s2 = xy2[:,2] # z
x2 = (xy2[:,0] / s2).astype(int)
y2 = (xy2[:,1] / s2).astype(int)

mask = (x2 >= 0) & (x2 < W) & (y2 >= 0) & (y2 < H)
x2 = x2[mask]
y2 = y2[mask]
# x2 = np.clip(x2.astype(int), 0, W-1)
# y2 = np.clip(y2.astype(int), 0, H-1)

for x, y in zip(x2, y2):
    projected[y, x, :] = [0, 0, 255]
cv2.imwrite(f"{sample_id:06d}_prj.png", projected)
cv2.imshow('projected', projected); cv2.waitKey(0); cv2.destroyAllWindows()

xy2.shape=(122798, 3)


In [31]:
# import cv2
# import math

# # read cam2
# cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)

# world_x_max = boundary.maxX
# world_x_min = 7 # boundary.minX
# world_y_max = 10 # boundary.maxY
# world_y_min = -10 # boundary.minY

# output_width  = 608
# output_height = 608
# world_x_interval = (world_x_max - world_x_min) / output_width
# world_y_interval = (world_y_max - world_y_min) / output_height
# # print(f"{world_x_interval=}")
# # print(f"{world_y_interval=}")

# def generate_direct_backward_mapping(extrinsic, intrinsic):
#     world_x_coords = np.arange(world_x_max, world_x_min, -world_x_interval)
#     world_y_coords = np.arange(world_y_max, world_y_min, -world_y_interval)
    
#     output_height = len(world_x_coords)
#     output_width = len(world_y_coords)
    
#     map_x = np.zeros((output_height, output_width)).astype(np.float32)
#     map_y = np.zeros((output_height, output_width)).astype(np.float32)
    
#     for i, world_x in enumerate(world_x_coords):
#         for j, world_y in enumerate(world_y_coords):
#             # world_coord : [world_x, world_y, 0, 1]
#             # uv_coord : [u, v, 1]
            
#             world_coord = [world_x, world_y, 0, 1]
#             camera_coord = extrinsic[:3, :] @ world_coord
#             uv_coord = intrinsic[:3, :3] @ camera_coord
#             uv_coord /= uv_coord[2]

#             # map_x : (H, W)
#             # map_y : (H, W)
#             # dst[i][j] = src[ map_y[i][j] ][ map_x[i][j] ]
#             map_x[i][j] = uv_coord[0]
#             map_y[i][j] = uv_coord[1]
            
#     return map_x, map_y

# p2 = np.array(list(map(float, calib_dict['P2']))).reshape(3, 4)
# # print(f"{p2=}, {p2.shape}")
# r0 = np.eye(4)
# r0[:3, :3] = np.array(list(map(float, calib_dict['R0_rect']))).reshape(3, 3)
# # print(f"{r0=}, {r0.shape}")
# tr = np.eye(4)
# tr[:3, :4] = np.array(
#     list(map(float, calib_dict['Tr_velo_to_cam']))).reshape(3, 4)
# # print(f"{tr=}, {tr.shape}")

# extrinsic = r0@tr
# intrinsic = p2

# map_x, map_y = generate_direct_backward_mapping(extrinsic, intrinsic)

# def remap_nearest(src, map_x, map_y):
#     src_height = src.shape[0]
#     src_width = src.shape[1]
    
#     dst_height = map_x.shape[0]
#     dst_width = map_x.shape[1]
#     dst = np.zeros((dst_height, dst_width, 3)).astype(np.uint8)
#     for i in range(dst_height):
#         for j in range(dst_width):
#             src_y = int(np.round(map_y[i][j]))
#             src_x = int(np.round(map_x[i][j]))
#             if 0 <= src_y < src_height and 0 <= src_x < src_width:
#                 dst[i][j] = src[src_y, src_x, :]
#     return dst 

# output_image_nearest = remap_nearest(cim2, map_x, map_y)
# cv2.imshow("nearest", output_image_nearest); cv2.waitKey(0); cv2.destroyAllWindows()
# output_image = cv2.remap(cim2, map_x, map_y, cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT)
# cv2.imshow("output", output_image); cv2.waitKey(0); cv2.destroyAllWindows()

# # mask = (output_image > [0, 0, 0])
# # output_image = output_image.astype(np.float32)
# # output_image_nearest = output_image_nearest.astype(np.float32)
# # raise Exception()

# def bilinear_sampler(imgs, pix_coords):
#     """
#     Construct a new image by bilinear sampling from the input image.
#     Args:
#         imgs:                   [H, W, C]
#         pix_coords:             [h, w, 2]
#     :return:
#         sampled image           [h, w, c]
#     """
#     img_h, img_w, img_c = imgs.shape
#     pix_h, pix_w, pix_c = pix_coords.shape
#     out_shape = (pix_h, pix_w, img_c)

#     pix_x, pix_y = np.split(pix_coords, [1], axis=-1)  # [pix_h, pix_w, 1]
#     pix_x = pix_x.astype(np.float32)
#     pix_y = pix_y.astype(np.float32)

#     # Rounding
#     pix_x0 = np.floor(pix_x)
#     pix_x1 = pix_x0 + 1
#     pix_y0 = np.floor(pix_y)
#     pix_y1 = pix_y0 + 1

#     # Clip within image boundary
#     y_max = (img_h - 1)
#     x_max = (img_w - 1)
#     zero = np.zeros([1])

#     pix_x0 = np.clip(pix_x0, zero, x_max)
#     pix_y0 = np.clip(pix_y0, zero, y_max)
#     pix_x1 = np.clip(pix_x1, zero, x_max)
#     pix_y1 = np.clip(pix_y1, zero, y_max)

#     # Weights [pix_h, pix_w, 1]
#     wt_x0 = pix_x1 - pix_x
#     wt_x1 = pix_x - pix_x0
#     wt_y0 = pix_y1 - pix_y
#     wt_y1 = pix_y - pix_y0

#     # indices in the image to sample from
#     dim = img_w

#     # Apply the lower and upper bound pix coord
#     base_y0 = pix_y0 * dim
#     base_y1 = pix_y1 * dim

#     # 4 corner vertices
#     idx00 = (pix_x0 + base_y0).flatten().astype(np.int32)
#     idx01 = (pix_x0 + base_y1).astype(np.int32)
#     idx10 = (pix_x1 + base_y0).astype(np.int32)
#     idx11 = (pix_x1 + base_y1).astype(np.int32)

#     # Gather pixels from image using vertices
#     imgs_flat = imgs.reshape([-1, img_c]).astype(np.float32)
#     im00 = imgs_flat[idx00].reshape(out_shape)
#     im01 = imgs_flat[idx01].reshape(out_shape)
#     im10 = imgs_flat[idx10].reshape(out_shape)
#     im11 = imgs_flat[idx11].reshape(out_shape)

#     # Apply weights [pix_h, pix_w, 1]
#     w00 = wt_x0 * wt_y0
#     w01 = wt_x0 * wt_y1
#     w10 = wt_x1 * wt_y0
#     w11 = wt_x1 * wt_y1
#     output = w00 * im00 + w01 * im01 + w10 * im10 + w11 * im11
#     return output

# def remap_bilinear(image, map_x, map_y):
#     pix_coords = np.concatenate([np.expand_dims(map_x, -1), np.expand_dims(map_y, -1)], axis=-1)
#     bilinear_output = bilinear_sampler(image, pix_coords)
#     output = np.round(bilinear_output).astype(np.int32)
#     return output    

# output_image_bilinear = remap_bilinear(cim2, map_x, map_y)
# cv2.imshow("bilinear", output_image_bilinear); cv2.waitKey(0); cv2.destroyAllWindows()
# output_image = cv2.remap(cim2, map_x, map_y, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
# cv2.imshow("output", output_image); cv2.waitKey(0); cv2.destroyAllWindows()